Author: Matthew Viafora

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/brain-tumor/Brain Tumor.csv
/kaggle/input/brain-tumor/bt_dataset_t3.csv
/kaggle/input/brain-tumor/Brain Tumor/Brain Tumor/Image3202.jpg
/kaggle/input/brain-tumor/Brain Tumor/Brain Tumor/Image1153.jpg
/kaggle/input/brain-tumor/Brain Tumor/Brain Tumor/Image1465.jpg
/kaggle/input/brain-tumor/Brain Tumor/Brain Tumor/Image688.jpg
/kaggle/input/brain-tumor/Brain Tumor/Brain Tumor/Image233.jpg
/kaggle/input/brain-tumor/Brain Tumor/Brain Tumor/Image3561.jpg
/kaggle/input/brain-tumor/Brain Tumor/Brain Tumor/Image1166.jpg
/kaggle/input/brain-tumor/Brain Tumor/Brain Tumor/Image1975.jpg
/kaggle/input/brain-tumor/Brain Tumor/Brain Tumor/Image1263.jpg
/kaggle/input/brain-tumor/Brain Tumor/Brain Tumor/Image920.jpg
/kaggle/input/brain-tumor/Brain Tumor/Brain Tumor/Image1629.jpg
/kaggle/input/brain-tumor/Brain Tumor/Brain Tumor/Image1701.jpg
/kaggle/input/brain-tumor/Brain Tumor/Brain Tumor/Image695.jpg
/kaggle/input/brain-tumor/Brain Tumor/Brain Tumor/Image1273.jpg
/kaggle/input/brain-tu

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
from skimage.io import imread
from sklearn.model_selection import train_test_split
import keras
from keras import Sequential
from keras.applications import MobileNetV2
from keras.layers import Dense
from keras.preprocessing import image

**Put first two columns into a dataframe:**

In [3]:
df = pd.read_csv('/kaggle/input/brain-tumor/Brain Tumor.csv',usecols=[0,1])
df.head()

,Image,Class
0,Image1,0
1,Image2,0
2,Image3,1
3,Image4,1
4,Image5,0


**Dataset Insight:**

Check for null values:

In [4]:
df.isnull().sum()

Image    0
Class    0
dtype: int64

Check for values to account for imbalances:

In [5]:
df['Class'].value_counts()

0    2079
1    1683
Name: Class, dtype: int64

Clean and Prepare Data:

In [6]:
path_list = []
base_path = "/kaggle/input/brain-tumor/Brain Tumor/Brain Tumor"
for i in os.listdir(base_path):
    path_list.append(os.path.join(base_path, i))

In [7]:
path_dict = {os.path.splitext(os.path.basename(x))[0]: x for x in path_list}
df['Paths'] = df['Image'].map(path_dict.get)

In [8]:
df.head()

,Image,Class,Paths
0,Image1,0,/kaggle/input/brain-tumor/Brain Tumor/Brain Tu...
1,Image2,0,/kaggle/input/brain-tumor/Brain Tumor/Brain Tu...
2,Image3,1,/kaggle/input/brain-tumor/Brain Tumor/Brain Tu...
3,Image4,1,/kaggle/input/brain-tumor/Brain Tumor/Brain Tu...
4,Image5,0,/kaggle/input/brain-tumor/Brain Tumor/Brain Tu...


Resize all images:

In [9]:
from PIL.Image import open
df["Pixels"]=df["Paths"].map(lambda x: np.asarray(open(x).resize((224,224))))

In [10]:
df.head()

,Image,Class,Paths,Pixels
0,Image1,0,/kaggle/input/brain-tumor/Brain Tumor/Brain Tu...,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ..."
1,Image2,0,/kaggle/input/brain-tumor/Brain Tumor/Brain Tu...,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ..."
2,Image3,1,/kaggle/input/brain-tumor/Brain Tumor/Brain Tu...,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ..."
3,Image4,1,/kaggle/input/brain-tumor/Brain Tumor/Brain Tu...,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ..."
4,Image5,0,/kaggle/input/brain-tumor/Brain Tumor/Brain Tu...,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ..."


**Preprocessing**

In [11]:
image_list = []
for i in range(0,len(df)):
    brain_img = df["Pixels"][i].astype(np.float32)
    img_array = image.img_to_array(brain_img)
    # Append image to a list of all images
    image_list.append(keras.applications.mobilenet_v2.preprocess_input(img_array))
    
# Convert image list to single array
input_x = np.array(image_list)

In [12]:
print(input_x.shape)
df.info()

(3762, 224, 224, 3)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3762 entries, 0 to 3761
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Image   3762 non-null   object
 1   Class   3762 non-null   int64 
 2   Paths   3762 non-null   object
 3   Pixels  3762 non-null   object
dtypes: int64(1), object(3)
memory usage: 117.7+ KB


In [13]:
label_y = np.array(df.Class)
label_y

array([0, 0, 1, ..., 0, 0, 0])

**Split Dataset into training set (80%) and testing set (20%)**

In [14]:
x_train, x_test, y_train, y_test = train_test_split(input_x, label_y, test_size = 0.2, random_state = 0)

In [15]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(3009, 224, 224, 3)
(753, 224, 224, 3)
(3009,)
(753,)


**Build modelnet model**

In [16]:
num_classes = 1
model = Sequential()
model.add(MobileNetV2(input_shape=(224,224,3), weights = "imagenet", include_top = False))
model.add(keras.layers.GlobalAveragePooling2D())
model.add(Dense(num_classes, activation='sigmoid', name = 'preds'))
model.layers[0].trainable=False

9412608/9406464 [==============================] - 0s 0us/step


In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Functi (None, 7, 7, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
preds (Dense)                (None, 1)                 1281      
Total params: 2,259,265
Trainable params: 1,281
Non-trainable params: 2,257,984
_________________________________________________________________


In [18]:
model.compile(
    loss = keras.losses.binary_crossentropy,
    optimizer=keras.optimizers.SGD(lr=0.1),
    metrics=['accuracy']
)

In [19]:
model.fit(
    x_train,
    y_train,
    epochs=50,
    verbose=1,
    validation_data=(x_test,y_test)
)

Epoch 1/50
95/95 [==============================] - 12s 59ms/step - loss: 2.5325 - accuracy: 0.6793 - val_loss: 0.6189 - val_accuracy: 0.8632
Epoch 2/50
95/95 [==============================] - 4s 39ms/step - loss: 0.6032 - accuracy: 0.8571 - val_loss: 0.9607 - val_accuracy: 0.7742
Epoch 3/50
95/95 [==============================] - 4s 40ms/step - loss: 0.6446 - accuracy: 0.8379 - val_loss: 0.5851 - val_accuracy: 0.8513
Epoch 4/50
95/95 [==============================] - 4s 39ms/step - loss: 0.5396 - accuracy: 0.8772 - val_loss: 0.5481 - val_accuracy: 0.8632
Epoch 5/50
95/95 [==============================] - 4s 39ms/step - loss: 0.5029 - accuracy: 0.8662 - val_loss: 0.3804 - val_accuracy: 0.8911
Epoch 6/50
95/95 [==============================] - 4s 41ms/step - loss: 0.4114 - accuracy: 0.8840 - val_loss: 0.6711 - val_accuracy: 0.8592
Epoch 7/50
95/95 [==============================] - 4s 40ms/step - loss: 0.4135 - accuracy: 0.8843 - val_loss: 0.6853 - val_accuracy: 0.8234
Epoch 8/50
9

In [20]:
model.save("model_brain.h5")
pretrained_cnn = keras.models.load_model('./model_brain.h5')

# evaluate model on holdout set
eval_score = pretrained_cnn.evaluate(x_test,y_test)
# print loss score
print('Eval loss:',eval_score[0])
# print accuracy score
print('Eval accuracy:',eval_score[1] )

24/24 [==============================] - 1s 30ms/step - loss: 0.2914 - accuracy: 0.9097
Eval loss: 0.2913912236690521
Eval accuracy: 0.9096945524215698
